- Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

- cd into your exercises folder for this module and run echo env.py >> .gitignore

- Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

- Use your function to obtain a connection to the employees database.

- Once you have successfully run a query:

- a. Intentionally make a typo in the database url. What kind of error message do you see?

- b. Intentionally make an error in your SQL query. What does the error message look like?

In [1]:
import env
import pandas as pd

db = 'employees'

url = env.db_url(db)

In [2]:
# 5. Successfully run a query

pd.read_sql('select * from employees limit 1', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26


In [12]:
# 6. Read the employees and titles tables into two separate DataFrames.

employees = pd.read_sql('SELECT * FROM employees', url)

In [13]:
titles = pd.read_sql('SELECT * FROM titles', url)

In [5]:
# 7. How many rows and columns do you have in each DataFrame?

employees.shape # (300024, 6)

(300024, 6)

In [6]:
titles.shape # (443308, 4)

(443308, 4)

In [19]:
# 8. Display the summary statistics for each DataFrame.

employees.describe()

# employees.drop(columns='emp_no').describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [17]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


In [14]:
# 9. How many unique titles are in the titles DataFrame?

titles['title'].value_counts().count() # 7

7

In [22]:
# 10. What is the oldest date in the to_date column?

titles.to_date.min() # 1985-03-01

Timestamp('1985-03-01 00:00:00')

In [21]:
# 11. Convert the 'to_date' column to datetime
titles['to_date'] = pd.to_datetime(titles['to_date'], errors='coerce')

# errors = 'coerce' turns any errors into NaT (Not a time)

titles['to_date'].max()

Timestamp('2002-08-01 00:00:00')